In [1]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [14]:
data = pd.read_csv('./parameters.csv', header=0,  index_col=0)
exps = pd.DataFrame(data)

exps.head(100)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
0,1,2023-08-21 13:21:55.6321388,2023-08-21 13:51:55.6334369,unl,reference,8,6,12,4,8,2,0.25,1.0,1692624115,1692625915
1,13,2023-08-21 20:04:36.2280356,2023-08-21 20:34:36.3287578,unl,d,24,6,12,4,8,2,0.25,1.0,1692648276,1692650076
2,16,2023-08-21 21:45:15.5920510,2023-08-21 22:15:15.5957340,unl,d,12,6,12,4,8,2,0.25,1.0,1692654315,1692656115
3,19,2023-08-21 23:25:55.7865751,2023-08-21 23:55:55.7882012,unl,d,6,6,12,4,8,2,0.25,1.0,1692660355,1692662155
4,22,2023-08-22 01:06:36.0191246,2023-08-22 01:36:36.0204104,unl,dlo,8,2,12,4,8,2,0.25,1.0,1692666396,1692668196
5,25,2023-08-22 02:47:17.7191035,2023-08-22 03:17:17.7224088,unl,dlo,8,3,12,4,8,2,0.25,1.0,1692672437,1692674237
6,27,2023-08-22 03:54:24.6930971,2023-08-22 04:24:24.6954750,unl,dlo,8,8,12,4,8,2,0.25,1.0,1692676464,1692678264
7,30,2023-08-22 05:35:05.7800299,2023-08-22 06:05:05.7934261,unl,dhi,8,6,6,4,8,2,0.25,1.0,1692682505,1692684305
8,33,2023-08-22 07:15:46.9664994,2023-08-22 07:45:47.0692861,unl,dhi,8,6,24,4,8,2,0.25,1.0,1692688546,1692690347
9,38,2023-08-22 10:03:34.7446053,2023-08-22 10:33:34.7460834,unl,dhi,8,6,8,4,8,2,0.25,1.0,1692698614,1692700414


In [6]:
data = pd.read_csv('./tables/overhead.csv', header=0,  index_col=0)
overhead = pd.DataFrame(data)

overhead.head(10)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes,bandwidth,propagationTime,stddev_propagationTime,messageOverhead,stddev_messageOverhead
0,2,unl,8,6,12,4,8,2,0.25,1.0,23,349.473952,1451.646409,1050.668381,4.369040e+05,591902.115969
1,3,unl,8,6,12,4,8,2,0.25,0.5,23,413.648073,986.740298,20.491311,5.217123e+05,207294.360093
2,6,unl,8,6,12,4,8,2,0.25,30.0,23,1056.993541,1325.467710,93.967445,1.412220e+06,221307.397559
3,9,unl,8,6,12,4,8,2,0.25,3.0,23,744.672331,1294.990615,294.123534,9.942037e+05,90174.581792
4,12,unl,24,6,12,4,8,2,0.25,1.0,23,824.463965,1315.527934,262.677875,1.086024e+06,657810.889190
5,15,unl,12,6,12,4,8,2,0.25,1.0,23,961.139534,1524.225677,51.515811,1.201043e+06,208678.232348
6,18,unl,6,6,12,4,8,2,0.25,1.0,23,621.270477,3237.055361,2525.238950,7.683017e+05,65413.322568
7,21,unl,8,2,12,4,8,2,0.25,1.0,23,604.560808,1788.039572,306.026796,7.785740e+05,500522.442187
8,24,unl,8,3,12,4,8,2,0.25,1.0,23,360.859540,1597.645548,326.575399,4.573593e+05,344040.074482
9,27,unl,8,8,12,4,8,2,0.25,1.0,23,287.544853,1157.719611,261.649231,3.345777e+05,360148.453783


In [7]:
consolidated_index = overhead['identifier']

print(consolidated_index)

0       2
1       3
2       6
3       9
4      12
5      15
6      18
7      21
8      24
9      27
10     30
11     33
12     36
13     39
14     42
15     45
16     48
17     51
18     54
19     57
20     60
21     63
22     66
23     69
24     77
25     78
26     81
27     84
28     87
29     90
30     93
31     96
32     99
33    102
34    105
35    108
36    111
37    114
38    117
39    120
40    123
41    126
42    129
43    132
44    135
45    138
46    141
47    144
Name: identifier, dtype: int64


In [12]:
exps_identifier = exps['experiment']

print(exps_identifier.astype(int))

0       1
1      13
2      16
3      19
4      22
5      25
6      27
7      30
8      33
9      38
10     39
11     43
12     46
13     48
14     53
15     56
16     57
17     60
18     63
19     66
20     70
21     72
22     75
23     78
24     81
25     88
26     90
27     93
28     97
29    100
30    102
31    105
32    108
33    115
34    117
35    120
36    123
37    127
38    129
39    134
40    135
41    138
42    141
43    144
44    147
45    151
46    153
47    156
48    159
49    161
50    163
51    166
52    168
53    171
54    173
55    178
Name: experiment, dtype: int64


In [35]:
data = pd.read_csv('./tables/timeSeries.csv', header=0,  index_col=0)
timeseries = pd.DataFrame(data)
ids = pd.DataFrame()
ids["experiment"] = timeseries["experiment"].drop_duplicates()

ids.head(10)

,experiment
0,152
1154,300
3644,301
3680,307
3702,309
4119,310
4591,328
7774,329
11868,330
15464,331


In [36]:
def experiment(filepath):
    # Retrieve experiments data from csv
    data = pd.read_csv(filepath, header=None)
    df = pd.DataFrame(data)

    #Rename columns
    experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

    #Correct timestamp
    experiments["start"] = experiments["start"].str.slice(0, 27)
    experiments["end"] = experiments["end"].str.slice(0, 27)

    #String to timestamp
    # experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    # experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
    experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


    experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
    experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

    #Drop fields we don't mneed for the moment
    exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

    #Get times for different intervals
    # intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
    # intervals.head(10)

    # expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
    # expTime['experiment'] = expTime.index
    expTime = exp.reset_index().rename({'index':'experiment'}, axis = 'columns')

    return expTime

experiments = experiment('../experiments.csv')
experiments.head(100)

/tmp/ipykernel_112580/2041158477.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
/tmp/ipykernel_112580/2041158477.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
0,0,2023-08-21 12:48:22.3741823,2023-08-21 13:18:22.3952804,unl,reference,8,6,12,4,8,2,0.25,1.0,1692622102,1692623902
1,1,2023-08-21 13:21:55.6321388,2023-08-21 13:51:55.6334369,unl,reference,8,6,12,4,8,2,0.25,1.0,1692624115,1692625915
2,2,2023-08-21 13:55:28.9783578,2023-08-21 14:25:28.9820206,unl,reference,8,6,12,4,8,2,0.25,1.0,1692626128,1692627928
3,3,2023-08-21 14:29:02.5866046,2023-08-21 14:59:02.6871797,unl,interval,8,6,12,4,8,2,0.25,0.5,1692628142,1692629942
4,4,2023-08-21 15:02:35.9708968,2023-08-21 15:32:36.0272486,unl,interval,8,6,12,4,8,2,0.25,0.5,1692630155,1692631956
5,5,2023-08-21 15:36:09.3380555,2023-08-21 16:06:09.3391894,unl,interval,8,6,12,4,8,2,0.25,0.5,1692632169,1692633969
6,6,2023-08-21 16:09:43.1463718,2023-08-21 16:39:43.1814633,unl,interval,8,6,12,4,8,2,0.25,30.0,1692634183,1692635983
7,7,2023-08-21 16:43:16.7045785,2023-08-21 17:13:16.7785160,unl,interval,8,6,12,4,8,2,0.25,30.0,1692636196,1692637996
8,8,2023-08-21 17:16:50.1424767,2023-08-21 17:46:50.1924402,unl,interval,8,6,12,4,8,2,0.25,30.0,1692638210,1692640010
9,9,2023-08-21 17:50:23.0310174,2023-08-21 18:20:23.0333186,unl,interval,8,6,12,4,8,2,0.25,3.0,1692640223,1692642023


In [37]:
parameters = ids.merge(experiments, on=["experiment"], how="left")

parameters.tail(100)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
79,220,2023-08-27 06:53:22.6371734,2023-08-27 07:23:22.6393613,unl,gossipFactor,8,6,12,4,8,2,1.00,1.00,1693119202,1693121002
80,227,2023-08-28 12:43:50.4559273,2023-08-28 13:13:50.4587305,general,reference,8,6,12,4,8,2,0.25,1.00,1693226630,1693228430
81,358,2023-09-14 12:51:45.4083755,2023-09-14 13:21:45.4113948,general,reference,8,6,12,4,8,2,0.25,1.00,1694695905,1694697705
82,359,2023-09-14 13:25:18.4306616,2023-09-14 13:55:18.4370965,general,reference,8,6,12,4,8,2,0.25,1.00,1694697918,1694699718
83,360,2023-09-14 13:58:51.2227549,2023-09-14 14:28:51.2271605,general,reference,8,6,12,4,8,2,0.25,1.00,1694699931,1694701731
84,361,2023-09-14 14:32:24.3331501,2023-09-14 15:02:24.3346556,general,reference,8,6,12,4,8,2,0.25,1.00,1694701944,1694703744
85,362,2023-09-14 15:05:57.1265995,2023-09-14 15:35:57.1282354,general,reference,8,6,12,4,8,2,0.25,1.00,1694703957,1694705757
86,363,2023-09-14 15:39:31.2180260,2023-09-14 16:09:31.2199064,general,reference,8,6,12,4,8,2,0.25,1.00,1694705971,1694707771
87,228,2023-08-28 13:17:25.0094743,2023-08-28 13:47:25.0663662,general,interval,8,6,12,4,8,2,0.25,0.50,1693228645,1693230445
88,229,2023-08-28 13:50:58.9294778,2023-08-28 14:20:58.9480353,general,interval,8,6,12,4,8,2,0.25,0.50,1693230658,1693232458


In [42]:
identifiers = timeseries.drop(columns=['type', '_time', 'count']).drop_duplicates()
parameters_final = parameters.merge(identifiers, on=['experiment']).drop(columns=["parameter","experiment", "start", "end", "startUnix", "endUnix"]).drop_duplicates().reset_index().drop(columns=['index'])

parameters_final.to_csv('./parameters.csv')
parameters_final.head(100)

# identifiers.head(300)

,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,identifier
0,unl,8,6,12,4,8,2,0.25,1.00,1
1,unl,8,6,12,4,8,2,0.25,0.50,13
2,unl,8,6,12,4,8,2,0.25,30.00,16
3,unl,8,6,12,4,8,2,0.25,3.00,19
4,unl,24,6,12,4,8,2,0.25,1.00,22
5,unl,12,6,12,4,8,2,0.25,1.00,25
6,unl,6,6,12,4,8,2,0.25,1.00,27
7,unl,8,2,12,4,8,2,0.25,1.00,30
8,unl,8,3,12,4,8,2,0.25,1.00,33
9,unl,8,8,12,4,8,2,0.25,1.00,38
